In [1]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append('../scripts')
import ml_gwas

In [2]:
reload(ml_gwas)
ensemble, df_features, df_amr = ml_gwas.train_ensemble(
    allele_table_path='../sample_data/strain_by_allele_binary_SA.csv.gz', 
    amr_path='../sample_data/strain_by_amr_SA.csv.gz', 
    antibiotic='ciprofloxacin', 
    gene_path='../sample_data/strain_by_gene_SA.csv.gz',
    core_cutoff=10, bootstrap_instances=0.8, bootstrap_features=0.5, 
    num_models=500)

Non-core genes: (288, 2968)
Core-gene alleles: (288, 20390)
Feature table: (288, 23358)
Iteration 50
Iteration 100
Iteration 150
Iteration 200
Iteration 250
Iteration 300
Iteration 350
Iteration 400
Iteration 450
Iteration 500


In [3]:
df_coefs = ensemble.get_coefficient_matrix(feature_names=df_features.columns,reduced=True, order=1)
df_coefs.head(10)

,Model_1,Model_2,Model_3,Model_4,Model_5,Model_6,Model_7,Model_8,Model_9,Model_10,...,Model_491,Model_492,Model_493,Model_494,Model_495,Model_496,Model_497,Model_498,Model_499,Model_500
Cluster_2151_Allele_0,NaN,NaN,0.000000,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,...,NaN,0.399707,0.000000,NaN,NaN,NaN,0.346903,NaN,0.000000,NaN
Cluster_159,0.236555,1.250636,NaN,0.000000,NaN,0.348291,NaN,NaN,NaN,NaN,...,NaN,NaN,0.165287,0.394535,0.000000,0.000000,NaN,0.593252,0.184598,NaN
Cluster_3356,NaN,NaN,0.000000,NaN,NaN,0.063643,0.003547,1.290868,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.095948,0.310999,NaN,NaN,NaN
Cluster_1782_Allele_0,NaN,NaN,0.271146,NaN,NaN,0.000000,0.230202,0.000000,0.0,NaN,...,0.0,0.137322,0.058060,NaN,NaN,NaN,NaN,NaN,NaN,0.0
Cluster_92_Allele_18,NaN,0.000000,0.465407,0.000000,NaN,NaN,0.000000,NaN,0.0,NaN,...,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.002830,NaN,NaN
Cluster_2469,0.085526,0.045688,NaN,NaN,NaN,NaN,0.099353,0.000000,NaN,0.084333,...,NaN,NaN,NaN,0.168764,NaN,0.009633,0.016732,NaN,NaN,NaN
Cluster_2974,NaN,0.022646,0.291166,0.019204,NaN,0.232343,NaN,NaN,NaN,NaN,...,NaN,NaN,0.825568,NaN,0.237453,0.014706,0.029007,NaN,0.128205,NaN
Cluster_394,0.232155,NaN,0.000000,NaN,0.030286,0.000000,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.221706,0.000000,0.000000,NaN,NaN,NaN,NaN
Cluster_671,0.257653,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,0.000000,0.289011,0.180199,NaN,NaN,NaN,0.000000,0.000000,0.0
Cluster_1314_Allele_0,0.000000,0.000000,NaN,0.478650,NaN,0.000000,0.327745,0.000000,NaN,NaN,...,0.0,0.000000,NaN,0.000000,NaN,0.019273,0.000000,0.000000,NaN,0.0


In [4]:
df_coefs = ensemble.get_coefficient_matrix(feature_names=df_features.columns,reduced=True, order=-1)
df_coefs.head(10)

,Model_1,Model_2,Model_3,Model_4,Model_5,Model_6,Model_7,Model_8,Model_9,Model_10,...,Model_491,Model_492,Model_493,Model_494,Model_495,Model_496,Model_497,Model_498,Model_499,Model_500
Cluster_126_Allele_1,NaN,NaN,NaN,NaN,-1.175549,NaN,NaN,NaN,-1.264790,NaN,...,-1.002310,-1.209870,NaN,NaN,-0.426223,-1.555585,-1.626465,-1.168619,-0.668526,-1.465042
Cluster_279_Allele_0,NaN,NaN,0.000000,-1.472172,-0.211421,-0.892603,-0.404869,NaN,-0.526927,NaN,...,NaN,0.000000,0.000000,-0.359399,NaN,NaN,0.000000,-0.410297,-0.637606,NaN
Cluster_2617_Allele_0,NaN,-0.598528,-0.131031,NaN,-0.507601,NaN,0.000000,0.000000e+00,-0.102103,-2.325538e-01,...,-0.311875,NaN,NaN,-0.361623,-0.003218,0.000000,0.000000,NaN,NaN,NaN
Cluster_1696_Allele_0,NaN,NaN,-0.109571,-0.005066,NaN,-0.133020,0.000000,NaN,-0.068101,-2.071165e-01,...,-0.280064,-0.060847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cluster_2243_Allele_0,-0.736188,-0.012251,-0.135006,NaN,NaN,-0.206673,0.000000,NaN,NaN,-3.817634e-07,...,-0.312612,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
Cluster_1028_Allele_0,-0.260651,NaN,0.000000,0.000000,0.000000,NaN,-0.471796,-6.890056e-07,0.000000,NaN,...,NaN,-0.152529,0.000000,-0.348114,-0.064811,0.000000,NaN,NaN,NaN,NaN
Cluster_1780_Allele_0,NaN,-0.007736,0.000000,0.000000,0.000000,0.000000,-0.447299,0.000000e+00,NaN,-4.804744e-01,...,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000
Cluster_92_Allele_22,-0.628059,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000e+00,0.000000,NaN,...,0.000000,-0.214855,0.000000,NaN,-0.354170,0.000000,0.000000,NaN,0.000000,NaN
Cluster_2412_Allele_5,NaN,0.000000,NaN,0.000000,0.000000,NaN,NaN,0.000000e+00,NaN,NaN,...,0.000000,-0.080955,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000
Cluster_4362_Allele_0,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,0.000000e+00,0.000000,NaN,...,NaN,NaN,-1.043532,NaN,-0.001318,0.000000,NaN,NaN,NaN,0.000000
